In [0]:
!pip install git+https://github.com/fastai/fastai@2e1ccb58121dc648751e2109fc0fbf6925aa8887

In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [0]:
from fastai.imports import *
from fastai.structured import *

In [0]:
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics

In [0]:
#Load the driver helper and mount
from google.colab import drive #permite usar archivos de drive

#This will prompt for authorization
drive.mount('/content/gdrive')

In [0]:
#lists the content of your google drive
!ls gdrive/'My Drive'/'Lesson 1'

In [0]:
!ls

In [0]:
!head gdrive/'My Drive'/'Lesson 1'/Train.csv

In [0]:
PATH="gdrive/My Drive/Lesson 1/"

In [0]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates=['saledate'])

In [11]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
saledate                    401125 non-null datetime64[ns]
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                

In [0]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

add_datepart(df_raw, 'saledate')    # la función add_datepart  crea nuevas columnas, cada una con una parte del campo original de tipo fecha


In [13]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


In [0]:
train_cats(df_raw)


In [15]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [0]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)
df_raw.UsageBand = df_raw.UsageBand.cat.codes
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')
import feather
df_raw = feather.read_dataframe('tmp/bulldozers-raw')


In [0]:
df, y, nas = proc_df(df_raw, 'SalePrice')

In [20]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9830601685729056

In [22]:
#df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')

def split_vals(a,n): 
  return a[:n], a[n:]

n_valid = 12000 # igual al tamaño del test data set de Kaggle
n_trn = len(df_trn)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

((389125, 66), (389125,), (12000, 66))

In [0]:
# Definimos algunas funciones utiles para nuestra revision de datos

def rmse(x,y): 
  return math.sqrt(((x-y)**2).mean())            #definimos una función que calcule el RMSE

def print_score(m):                                           #definimos una función para que imprima rmse y score de entrenamiento y validación
    res = ["--rmse train: ", rmse(m.predict(X_train), y_train),  "--rmse valid: ",rmse(m.predict(X_valid), y_valid),
                "--m.score train:", m.score(X_train, y_train), "--m.score valid: ", m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): 
      res.append ("obb_score: ")
      res.append(m.oob_score_)
    print(res)
    
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [0]:
df_raw        #revisamos los datos

In [0]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 1min 46s, sys: 124 ms, total: 1min 46s
Wall time: 55 s
['--rmse train: ', 0.09067076663590509, '--rmse valid: ', 0.2513205365624304, '--m.score train:', 0.9828181706368858, '--m.score valid: ', 0.8872013425853322]
